# Estimating the stock beta via python

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pandas_datareader.data as web
import datetime as dt
import statsmodels.api as sm

sns.set()
plt.style.use('dark_background')

In [8]:
end = dt.date(2020,6,30)
start = dt.date(end.year - 5, end.month, end.day)
funds = ['^FDGRX'] # Fidelity Growth Company (FDGRX)

In [9]:
fundsret = web.DataReader(funds, data_source='yahoo', start=start, end=end)['Adj Close'].pct_change()
# fundsret = web.get_data_yahoo(funds,start,end)['Adj Close'].pct_change()
fundsret

c:\users\hugo\appdata\local\programs\python\python37\lib\site-packages\pandas_datareader\base.py:272: SymbolWarning: Failed to read symbol: '^FDGRX', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


RemoteDataError: No data fetched using 'YahooDailyReader'

## Monthly Returns

In [ ]:
monthly_rets = df.resample('M').agg(lambda x: (x+1).prod() - 1)
monthly_rets = monthly_rets[1:]
monthly_rets.head()

## Bulding the regression model

In [ ]:
X = monthly_rets['^GSPC']
y = monthly_rets['MSFT']

# the intercept is not defined as default
X_sm = sm.add_constant(X)

In [ ]:
X.shape, y.shape

In [ ]:
model = sm.OLS(y, X_sm)
results = model.fit()
results.summary()

Alfa = coef X const = 0,0191

Beta = coef X ^GSPC = 0,7896

In [ ]:
fig, ax = plt.subplots(figsize=(21,9))
sns.regplot(x='^GSPC', y='MSFT', data=monthly_rets, ci=95, ax=ax)  #ci is the Confidence Interval
plt.show()

## Taking the risk free rate

In [ ]:
rf = reader.DataReader('F-F_Research_Data_Factors', 'famafrench',start,end)[0].RF
rf = rf[1:-1]
rf.head(5)

In [ ]:
rf.tail(5), rf.shape

In [ ]:
# Get the Excess Return

monthly_rets['MSFT-rf'], monthly_rets['^GSPC-rf'] = monthly_rets['MSFT'] - rf.values, monthly_rets['^GSPC'] - rf.values
monthly_rets.head()

## Bulding the regression model again

In [ ]:
X = monthly_rets['^GSPC-rf']
y = monthly_rets['MSFT-rf']

In [ ]:
X_sm = sm.add_constant(X)

In [ ]:
model = sm.OLS(y,X_sm)

In [ ]:
results = model.fit()
results.summary()

In [ ]:
fig, ax = plt.subplots(figsize=(21,9))
sns.regplot(x='^GSPC-rf', y='MSFT-rf', data=monthly_rets, ci=95, ax=ax)  #ci is the Confidence Interval
plt.show()